# `qp` Demo

In this notebook we use the `qp` module to approximate some standard 1-D PDFs using sets of quantiles, and assess the accuracy of the quantile parametrization(TM).

### Requirements

To run `qp`, you will need to first install the module. 

In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.interpolate as spi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import qp

## The `qp.PDF` Class

This is the basic element of `qp` - an object representing a probability density function. This class is stored in the module `pdf.py`.

In [ ]:
# ! cat qp/pdf.py

## Approximating a Gaussian

Let's summon a PDF object, and initialize it with a standard function - a Gaussian.

In [ ]:
dist = sps.norm(loc=0, scale=1)
P = qp.PDF(truth=dist)

Now, let's compute a set of evenly spaced quantiles. These will be carried by the `PDF` object as `p.quantiles`.  We also demonstrate the initialization of a 'PDF' object with quantiles and no truth function.

In [ ]:
quantiles = P.quantize(percent=10.)
Q = qp.PDF(quantiles=quantiles)

Let's also compute a histogram representation that will be carried by the 'PDF' object as 'p.histogram'.  We can similary initialize a 'PDF' object with a histogram and no truth function.

In [ ]:
T = qp.PDF(truth=dist)
histogram = T.histogramize(nbins=10, binrange=[-2., 2.])
R = qp.PDF(histogram=histogram)

Let's test interpolation at a single point for the quantile and histogram representations.

In [ ]:
print(P.approximate(0.314, using='quantiles'))
#print(P.approximate(0.314, using='histogram'))

Now, let's interpolate the function over an evenly spaced grid with points within and out of the quantile range.  

In [ ]:
grid = np.linspace(-3., 3., 100)
P.approximate(grid, using='quantiles')

Let's visualize the PDF object in order to compare the truth and the approximation.  The solid line shows the true PDF evaluated between the bounds.  The dashed lines show the 10 percentiles that we asked for. 10% of the total probability mass is enclosed between each pair. Note that the quantiles refer to the probability distribution *between the bounds*, because we are not able to integrate numerically over an infinite range.  The red, dotted line shows an interpolation (and extrapolation) based on the quantiles.

In [ ]:
bounds = (-3.0, 3.0)
P.plot(limits=bounds, points=grid)

T.plot(limits=bounds, points=grid)

Next, let's compare the quantile approximation to the truth using the Kullback-Leibler Divergence (KLD).  The KLD is a measure of how close two probability distributions are to one another -- a smaller value indicates closer agreement.  It is measured in units of bits of information, the information lost in going from the second distribution to the first distribution.  The KLD calculator here takes in a shared grid upon which to evaluate the true distribution and the interpolated quantile approximation of that distribution and returns the KLD of the approximation relative to the truth, which is not in general the same as the KLD of the truth relative to the approximation.  Below, we'll calculate the KLD of the approximation relative to the truth over different ranges, showing that it increases as it includes areas where the true distribution and interpolated distribution diverge.

In [ ]:
qD1 = qp.utils.calculate_kl_divergence(P, Q, limits=(-1.,1.))
qD2 = qp.utils.calculate_kl_divergence(P, Q, limits=(-2.,2.))
qD3 = qp.utils.calculate_kl_divergence(P, Q, limits=(-3.,3.))

hD1 = qp.utils.calculate_kl_divergence(T, R, limits=(-1.,1.))
hD2 = qp.utils.calculate_kl_divergence(T, R, limits=(-2.,2.))
hD3 = qp.utils.calculate_kl_divergence(T, R, limits=(-3.,3.))

print(qD1, qD2, qD3)
print(hD1, hD2, hD3)

The progression of KLD values should follow that of the root mean square (RMS), another measure of how close two functions are to one another.  The RMS also increases as it includes areas where the true distribution and interpolated distribution diverge.  Unlike the KLD, the RMS is symmetric, meaning the distance measured is not that of one distribution from the other but of the symmetric distance between them.

In [ ]:
qRMS1 = qp.utils.calculate_rms(P, Q, limits=(-1.,1.))
qRMS2 = qp.utils.calculate_rms(P, Q, limits=(-2.,2.))
qRMS3 = qp.utils.calculate_rms(P, Q, limits=(-3.,3.))

hRMS1 = qp.utils.calculate_rms(T, R, limits=(-1.,1.))
hRMS2 = qp.utils.calculate_rms(T, R, limits=(-2.,2.))
hRMS3 = qp.utils.calculate_rms(T, R, limits=(-3.,3.))

print(qRMS1, qRMS2, qRMS3)
print(hRMS1, hRMS2, hRMS3)